## Код для предварительного анализа данных датасета

In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objs as go

from datetime import date
from datetime import datetime, time

In [20]:
data = pd.read_csv('Table 1.csv')

data.head(5)

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-01-09 04:49:13 UTC,2.348768e+18,1.515966e+18,2.268105e+18,appliances.environment.water_heater,midea,78.68,NaN
1,2020-01-09 13:55:37 UTC,2.348768e+18,1.515966e+18,2.268105e+18,electronics.smartphone,oppo,231.23,NaN
2,2020-01-09 12:44:51 UTC,2.348768e+18,1.515966e+18,2.268105e+18,appliances.kitchen.microwave,samsung,120.35,NaN
3,2020-01-08 08:21:30 UTC,2.348768e+18,2.273948e+18,2.268105e+18,appliances.environment.water_heater,thermex,92.57,NaN
4,2020-01-08 08:21:30 UTC,2.348768e+18,1.515966e+18,NaN,57.87,NaN,NaN,NaN


Типы данных

In [21]:
data.dtypes

event_time        object
order_id         float64
product_id       float64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object

### Количество уникальных данных по категориям

In [22]:
data.nunique()

event_time       46573
order_id         48570
product_id        1609
category_id        395
category_code      274
brand              790
price              967
user_id            802
dtype: int64

In [23]:
data.set_index('order_id', inplace=True)

### Количество пустых ячеек

In [24]:
null_columns = data.columns[data.isnull().any()]
data[null_columns].isnull().sum()

category_id      22122
category_code    25394
brand            25235
price            22122
user_id          94622
dtype: int64

In [25]:
n_unique_products = data['product_id'].nunique()
n_unique_users = data['user_id'].nunique()
print('Количество уникальных пользователей: ' + str(n_unique_users) +'. Количество уникальных продуков: ' + str(n_unique_products))

Количество уникальных пользователей: 802. Количество уникальных продуков: 1609


In [26]:
data['event_time']=pd.to_datetime(data['event_time'])

In [27]:
data.dropna(subset=['category_code'],inplace=True)

In [28]:
data['category'] = data['category_code'].str.rsplit('.', n=1, expand=True)[1]
data.drop(columns=['category_code'], inplace=True)

### ТОП-10 брендов приносящих наибольшую прибыль

In [29]:
best_performing_brands = data.groupby('brand')['price'].sum().reset_index().sort_values('price', ascending=False).head(10)
print(best_performing_brands)

       brand       price
621  samsung  2276035.98
419    apple  1357056.80
546       lg   741918.23
514   huawei   564821.34
545   lenovo   261984.56
631     sony   238102.14
588     oppo   233194.84
424     asus   214966.23
445    bosch   204926.39
428      ava   180296.72


### Самые продаваемые бренды

In [30]:
most_sold_brands = data.groupby('brand')['price'].agg('count').reset_index().sort_values('price', ascending=False).head(10)
most_sold_brands.rename(columns={"brand": "brand", "price": "times_sold"}, inplace=True)
print(most_sold_brands)

        brand  times_sold
621   samsung       10245
514    huawei        3257
428       ava        2306
419     apple        2180
645     tefal        2094
546        lg        1575
588      oppo        1341
596   philips        1224
659       tvs        1221
450  camelion         916


### Самые продаваемые категории товаров

In [31]:
most_sold_categories = data.groupby('category')['price'].agg('count').reset_index().sort_values('price', ascending=False).head(10)
most_sold_categories.rename(columns={"category": "category", "price": "times_sold"}, inplace=True)
print(most_sold_categories)

          category  times_sold
159     smartphone       12855
173             tv        3093
174         vacuum        2468
128         kettle        2229
164          table        2195
150  refrigerators        1978
141       notebook        1919
178         washer        1890
121      headphone        1787
140          mouse        1362


### В какое время было совершено наибольшее количество покупок

In [32]:
best_time = data.groupby(data['event_time'].dt.strftime('%r'))['price'].sum().sort_values()[lambda x: x<= 900000].tail(10)
print(best_time)

event_time
10:17:20 AM    4763.58
11:08:42 AM    4772.67
11:42:52 AM    4882.22
12:37:34 PM    5156.13
08:15:01 AM    5195.18
11:20:06 AM    5536.70
04:54:24 AM    5648.06
12:44:47 PM    5691.89
12:34:08 PM    6220.53
10:52:12 AM    9428.20
Name: price, dtype: float64


### Самый прибыльный и наименее прибыльный месяцы

In [33]:
best_month = data.groupby(data['event_time'].dt.strftime('%B'))['price'].sum().sort_values()
print(best_month)

event_time
July          1345.16
January    8356902.91
Name: price, dtype: float64


### Время покупки пользователем и объем транзакций

In [34]:
user_purchase_time_GMV = data.groupby(data['event_time'].dt.strftime('%H'))['price'].sum().reset_index()
print(user_purchase_time_GMV)

   event_time       price
0          02        0.93
1          03     2695.94
2          04   151583.46
3          05   433927.73
4          06   627371.21
5          07   784901.28
6          08   836549.13
7          09   895522.16
8          10  1000775.51
9          11   958729.87
10         12   945246.50
11         13   828884.79
12         14   476848.87
13         15   310167.35
14         16    84171.79
15         17    17853.42
16         18     3018.13


### Процент данных, отсутствующих в каждом столбце

In [35]:
display(data.isna().sum()*100/data.shape[0])

event_time      0.000000
product_id      0.000000
category_id    29.651771
brand          30.994826
price          29.651771
user_id        95.237648
category        0.000000
dtype: float64

### В какие месяцы происходили события

In [36]:
# На доработке
def get_month_year(timestamp):
    '''
    Read timestamp and return year-month
    '''
    try:
        # 2020-04-24 11:50:39 UTC
        ft = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S %Z')
        return datetime.strftime(ft, "%Y-%m")
    except:
        print('Could not process timestamp')
        return None

data['year_month'] = data['event_time'].apply(get_month_year)
data.year_month.unique()